## Strategic Location using Facebook Analytics

If you were to open your own cafe, would you not want to effortlessly identify the most suitable location to set up your shop? Choosing an optimal physical location is a critical decision for numerous businesses, as many factors contribute to the final choice of the location.
Features selected:
* category of cafe
* category of neighboring cafes
* checkins of localities

In [12]:
#preprocessing
import pandas as pd
import numpy as np

def JsontoDataFrame(jsonFile):
    data=pd.read_json(jsonFile)
    data=data.as_matrix()
    data= data.transpose()
    data=pd.DataFrame(data,columns=['Category','Check-Ins','Latitude','Likes','Longitude','Name'])
    return data

def filterFoodRelated(data):
    totalIndex=0
    remIndex=[]
    keepIndex=[]
    uniqueCategories=[]
    frConstraints=['Diner','Fruit','Vegetable','Caterer','Bar','Pub','Grill','Hotel','Restaurant','Lounge','Pizza Place','Dessert Shop','Coffee','Food','Beverage','Cafe']
    
    for index,rows in data.iterrows():
        #print index,rows['Category']
        for cat in rows['Category']:
            categorysublist=cat.split(' ')
            if len(set(categorysublist).intersection(frConstraints))>0:
                keepIndex.append(index)
                uniqueCategories.append(cat)
                break
            
        totalIndex=index
    
    remIndex=list(set(range(totalIndex+1))-set(keepIndex))
    data=data.drop(data.index[remIndex]).reset_index(drop=True)
    return data,list(set(uniqueCategories))
    
all_data=JsontoDataFrame('fbData.json')
food_data,unique_categories=filterFoodRelated(AllData)

In [30]:
#Label Binarizer Encoding Categories
from sklearn import preprocessing
lb=preprocessing.LabelBinarizer()
lb.fit(unique_categories)

def orCategories(bcat1,bcat2):
    """
    Oring Category Codes 
    """
    result_cat=[]
    for val1,val2 in zip(bcat1,bcat2):
        if val1==1 or val2==1:
            result_cat.append(1)
        else:
            result_cat.append(0)
    return result_cat

def addBinaryCategoryData(data):
    """
    Finding Binary Category Codes of all entries
    """
    binary_category=[]
    for catlist in data["Category"]:
        final_category=[0]*len(lb.classes_)
        for cat in catlist:
            binary_encoded_cat=lb.transform([str(cat)])[0]
            finalcat=orCategories(final_category,binary_encoded_cat)
        binary_category.append(finalcat)
    return binary_category

food_data['Category Code']=pd.Series(addBinaryCategoryData(food_data))

In [59]:
#Finding Neigbouring Food related Joints
from math import radians, cos, sin, asin, sqrt

def haversine_dist(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    m = 6367 * c*1000
    return m


def findNeighbors(data,target_latitude,target_longitude,distance,is_food_related=True):
    """
    Finding Neighbors to a given point on map
    """
    neighbor_name=[]
    neighbor_category=[0]*len(lb.classes_)
    neighbor_checkins=0
    if is_food_related:
        for latitude,longitude,name,binary_category,check_in in zip(data['Latitude'],data['Longitude'],data['Name'],data['Category Code'],data['Check-Ins']):
            neighbor_latitide=latitude
            neighbor_longitude=longitude
            if haversine_dist(target_longitude,target_latitude,neighbor_longitude,neighbor_latitide) < distance and target_longitude!=neighbor_longitude and neighbor_latitide!=target_latitude:
                neighbor_name.append(name)
                neighbor_category=orCategories(neighbor_category,binary_category)
                neighbor_checkins=neighbor_checkins+check_in
        return neighbor_name,neighbor_category,neighbor_checkins
    else:
        for latitude,longitude,name,check_in in zip(data['Latitude'],data['Longitude'],data['Name'],data['Check-Ins']):
            neighbor_latitide=latitude
            neighbor_longitude=longitude
            if haversine_dist(target_longitude,target_latitude,neighbor_longitude,neighbor_latitide) < distance and target_longitude!=neighbor_longitude and neighbor_latitide!=target_latitude:
                neighbor_name.append(name)
                neighbor_checkins=neighbor_checkins+check_in
    return neighbor_name,neighbor_checkins
         
def findAllNeighbors(distance,is_food_related=True,*data):
    DISTANCE=distance
    all_neighbour_name=[]
    all_neighbour_category=[]
    all_neighbour_checkins=[]
    if is_food_related:
        for lat,lon in zip(data[0]['Latitude'],data[0]['Longitude']):
            neighbor_name,neighbor_category,neighbor_checkins=findNeighbors(data[0],lat,lon,DISTANCE,True)
            all_neighbour_name.append(neighbor_name)
            all_neighbour_category.append(neighbor_category)
            all_neighbour_checkins.append(neighbor_checkins)
    
    else:
        food_related_joints=data[0]['Name']
        for lat,lon,name in zip(data[1]['Latitude'],data[1]['Longitude'],data[1]['Name']):
            if name in food_related_joints:
                
                neighbor_name,neighbor_checkins=findNeighbors(data[1],lat,lon,DISTANCE)
                all_neighbour_name.append(neighbor_name)
                all_neighbour_checkins.append(neighbor_checkins)

    return all_neighbour_name,all_neighbour_category,all_neighbour_checkins

    

def findAverageNeighborCheckIns(data,is_food_related=True):
    average_neigbor_checkins=[]
    if is_food_related:
        column_total_check_ins='Food-Related Neighbor Total Check-Ins'
        column_neigbor_names='Food-Related Neighbor Names'
    else:
        column_total_check_ins='All Neighbor Total Check-Ins'
        column_neigbor_names='All Neighbor Names'
    for neigbors,check_ins in zip(data[column_neigbor_names],data[column_total_check_ins]):
        average_neigbor_checkins.append(check_ins/len(neigbors))
    return average_neigbor_checkins
    
    
    
all_neighbour_name,all_neighbour_category,all_neighbour_checkins=findAllNeighbors(1000,True,food_data,all_data) 


food_data['Food-Related Neighbor Names']=pd.Series(all_neighbour_name)
food_data['Food-Related Neighbor Category Codes']=pd.Series(all_neighbour_category)
food_data['Food-Related Neighbor Total Check-Ins']=pd.Series(all_neighbour_checkins)
#food_data.drop('Food-Related Neighbor Check-Ins', axis=1, inplace=True)
food_data['Food-Related Neighbor Average Check-Ins']=pd.Series(findAverageNeighborCheckIns(food_data))


In [60]:
all_neighbour_name,all_neighbour_category,all_neighbour_checkins=findAllNeighbors(1000,False,food_data,all_data) 



In [61]:

food_data['All Neighbor Names']=pd.Series(all_neighbour_name)
food_data['All Neighbor Category Codes']=pd.Series(all_neighbour_category)
food_data['All Neighbor Total Check-Ins']=pd.Series(all_neighbour_checkins)
food_data['All Neighbor Average Check-Ins']=pd.Series(findAverageNeighborCheckIns(food_data))

In [62]:
food_data

,Category,Check-Ins,Latitude,Likes,Longitude,Name,Category Code,Food-Related Neighbor Names,Food-Related Neighbor Category Codes,Food-Related Neighbor Total Check-Ins,Food-Related Neighbor Average Check-Ins,All Neighbor Names,All Neighbor Category Codes,All Neighbor Total Check-Ins,All Neighbor Average Check-Ins
0,[Bar & Grill],54551,19.0995,3348,72.9166,"R-city Mall, Ghatkopar","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Rajdhani, R City Mall, Poptate Vikroli, Fort,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, ...",57641,8234,NaN,NaN,NaN,8234
1,"[Barbecue Restaurant, Indian Restaurant]",6986,19.1839,964,72.9516,Barbeque Nation,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Urban Tadka, Timbuctoo Mulund R-Mall, Poptate...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",51014,8502,NaN,NaN,NaN,8502
2,[Indian Restaurant],2664,19.1837,36,72.9509,Urban Tadka,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Barbeque Nation, Timbuctoo Mulund R-Mall, Pop...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",57188,8169,NaN,NaN,NaN,8169
3,[Bar & Grill],13474,19.1199,3945,72.9079,Mirchi and Mime,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Breeze - Powai, Chili's, Hiranandani, Powai, ...","[0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",238404,18338,NaN,NaN,NaN,18338
4,[Food & Beverage],4041,19.1837,127,72.9507,Timbuctoo Mulund R-Mall,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[Barbeque Nation, Urban Tadka, Poptates,R mall...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",55811,7973,NaN,NaN,NaN,7973
5,[Food & Beverage Company],100,19.1129,3795,72.9179,Homely,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Timbuctoo, Breeze - Powai, Chili's, Hirananda...","[1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, ...",168385,16838,NaN,NaN,NaN,16838
6,"[Lounge, Italian Restaurant, Cafe]",32157,19.1109,3418,72.9179,Timbuctoo,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[Homely, Breeze - Powai, Poptate Vikroli]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",57128,19042,NaN,NaN,NaN,19042
7,[Coffee Shop],8624,19.1744,7178,72.9506,Love & Latte,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[Woodland Retreat, Mcdonalds, Mulund, Royal Tr...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...",17299,3459,NaN,NaN,NaN,3459
8,[Asian Fusion Restaurant],6995,19.1838,435,72.9513,"Poptates,R mall,mulund.( West)","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Barbeque Nation, Urban Tadka, Timbuctoo Mulun...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",51005,8500,NaN,NaN,NaN,8500
9,[Vegetarian/Vegan Restaurant],4330,19.1756,862,72.9437,Woodland Retreat,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Love & Latte, Bombay Bites Mulund, Royal Trea...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, ...",16878,3375,NaN,NaN,NaN,3375
